# Lightgbm com GPU

In [ ]:
!rm -r /opt/conda/lib/python3.7/site-packages/lightgbm
!git clone --recursive https://github.com/Microsoft/LightGBM

In [ ]:
!apt-get install -y -qq libboost-all-dev

In [ ]:
%%bash
cd LightGBM
rm -r build
mkdir build
cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)

In [ ]:
!cd LightGBM/python-package/;python3 setup.py install --precompile

In [ ]:
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!rm -r LightGBM

# Importa os dados

In [ ]:
import catboost
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
import lightgbm as lgb

In [ ]:
train_data = pd.read_csv("../input/enconder-do-catboost-playmar21/train_enconded.csv").drop("Unnamed: 0", axis = 1)

train_data

In [ ]:
test_data = pd.read_csv("../input/enconder-do-catboost-playmar21/test_enconded.csv").drop("Unnamed: 0", axis = 1)

test_data

# Fita os modelos

## Os Modelos

### CatBoost

In [ ]:
cv_dataset = catboost.Pool(data = train_data.drop(["target"], axis = 1), label = train_data["target"])

cat_model = catboost.CatBoostClassifier(
    eval_metric = "AUC",
    loss_function = "Logloss",
    task_type = "GPU",
    depth = 8,
    iterations = 1500,
    random_seed = 1,
    learning_rate = 0.1
)

### LightGBM

In [ ]:
lgb_model = lgb.LGBMClassifier(
    learning_rate = 0.1,
    n_estimators = 50,
    num_leaves = 16,
    boosting_type = "gbdt",
    objective = "binary",
    random_state = 501,
    colsample_bytree = 0.65,
    subsample = 0.75,
    reg_alpha = 1.2,
    reg_lambda = 1.2,
    device = "gpu",
    gpu_platform_id = 0,
    gpu_device_id = 0
)

### XGDBoost

In [ ]:
xgd_model = xgb.XGBClassifier(
    n_estimators = 50,
    max_depth = 6,
    objective = "binary:logistic",
    eval_metric = "auc",
    tree_method = "gpu_hist",
    use_label_enconder = False
)

## .fit e faz as predições

In [ ]:
voting_model = VotingClassifier(estimators = [("cat", cat_model), ("lgb", lgb_model), ("xgd", xgd_model)], voting = "hard")

voting_model.fit(train_data[train_data.columns[:-1]], train_data["target"])

In [ ]:
preds = voting_model.predict(test_data.drop("id", axis = 1))

preds

In [ ]:
submission = test_data[["id"]]
submission["target"] = preds

submission

In [ ]:
submission.to_csv("sub.csv", index = False)